# Pre-processing ISOX files from orbitrap


# Minimum

In [ ]:
def preproc_isox(path, df1_split_times,df2_split_times,df3_split_times,df4_split_times):
    import pandas as pd
    import os
    #read in the file from the specified path
    print(type(path))
    
    data_isox = pd.read_table(path, sep='\s+')
    
    #print info
    print('The total length of the file in minutes is from \n' + str(data_isox['time.min'].min())+' min to '+
      str(data_isox['time.min'].max()) + ' min')
    print('____________________________________')
    print('The chosed split of the file is into 4 files \n 1st ' 
        + str(df1_split_times) +
        ' min \n 2nd ' + str(df2_split_times) +
        ' min \n 3rd ' + str(df3_split_times) +
        ' min \n 4th ' + str(df4_split_times) +' min')

    #split the dataframe into 4 based on provided data
    df1 = data_isox[(data_isox['time.min'] > df1_split_times[0]) & (data_isox['time.min'] < df1_split_times[1])] 
    df2 = data_isox[(data_isox['time.min'] > df2_split_times[0]) & (data_isox['time.min'] < df2_split_times[1])]
    df3 = data_isox[(data_isox['time.min'] > df3_split_times[0]) & (data_isox['time.min'] < df3_split_times[1])]
    df4 = data_isox[(data_isox['time.min'] > df4_split_times[0]) & (data_isox['time.min'] < df4_split_times[1])]
    
    # reset times and scan number in the new dataframes
    dfs_list=[df1,df2,df3,df4]
    for df in dfs_list:
        df.loc[:,'time.min']=df['time.min']-df['time.min'].min()+0.01 
        df.loc[:,'scan.no']=df['scan.no']-df['scan.no'].min()+1

    # #write down the dataframes
    print('____________________________________')
    print('Current file is ' + path)
    print('Currently saving:')
    
    for df in dfs_list:
        outpath=df['filename'].iloc[0]
        os.makedirs(outpath, exist_ok=True)  
        name=(outpath +'/'+ df['filename'].iloc[0]+'_'+str(df['compound'].min())+'_'+str(df['compound'].max())+'.isox')
        
        print(name)
        df.to_csv(name, sep='\t',float_format='%.3f')
    
    print('New files are saved in ' + outpath)

In [ ]:
def main():
    import glob
    import pandas as pd
    import os
    
    paths=glob.glob(os.path.join('../data/',"*.isox"))
    #path=glob.glob(os.path.join("./*.isox"))
    # Set path where to save new files
    #outpath='../data/'

    # Set split times for the new files
    df1_split_times=[0,120]
    df2_split_times=[120.05,210]
    df3_split_times=[210.05,300]
    df4_split_times=[300.05,330]

    # this lines runs the function
    for path in paths:
        # do not process if folder exists 
        #print('path '+ path)
        #print((path.split('/')[2]).split('.')[0])
        fold_name=path.split('.')[0]
        isExist=os.path.exists('./'+fold_name)
        if not isExist:
            print('does not exist -> start processing')
            preproc_isox(path, df1_split_times,df2_split_times,df3_split_times,df4_split_times)
        else:
            print(fold_name +' already exists')
            continue
if __name__ == "__main__":
    main() 

In [ ]:
import glob
import os
#!ls
paths=glob.glob(os.path.join("*.isox"))
for path in paths:
    print(path)
    fold_name=path.split('.')[0]
    print(fold_name)

In [ ]:
(path.split('/')[2]).split('.')[0]

## missing compound string check

In [ ]:
#for every new file calculate percentage of 'lost' compounds


In [51]:
import pandas as pd
#path='../data/52_Cholesterol-Nudibranch-57_125-400.isox'
path='./94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_125-400.isox'
data_isox = pd.read_table(path, sep='\s+')
#split times
df1_split_times = [0,120]
df2_split_times = [120.05,210]
df3_split_times = [210.05,300]
df4_split_times = [300.05,330]
#split the dataframe into 4 based on provided data
df1 = data_isox[(data_isox['time.min'] > df1_split_times[0]) & (data_isox['time.min'] < df1_split_times[1])] 
df2 = data_isox[(data_isox['time.min'] > df2_split_times[0]) & (data_isox['time.min'] < df2_split_times[1])]
df3 = data_isox[(data_isox['time.min'] > df3_split_times[0]) & (data_isox['time.min'] < df3_split_times[1])]
df4 = data_isox[(data_isox['time.min'] > df4_split_times[0]) & (data_isox['time.min'] < df4_split_times[1])]
# reset times and scan number in the new dataframes
dfs_list = [df1,df2,df3,df4]
for df in dfs_list:
    df.loc[:,'time.min'] = df['time.min']-df['time.min'].min()+0.01 
    df.loc[:,'scan.no'] = df['scan.no']-df['scan.no'].min()+1

In [ ]:
df3.iloc[::2].head()

In [ ]:
#number of scans
df3['scan.no'].max()

In [ ]:
[df3[(df3['compound']==comp)]['isotopolog'].count()/2 for comp in compounds ]

In [ ]:
df3['compound'].unique()

In [ ]:
compounds=df3['compound'].unique()
#print compound is its quantity does not match to number of scans
prob_comps=[comp for comp in compounds if df3[(df3['compound']==comp)]['isotopolog'].count()/2 != df3['scan.no'].max()]
print(prob_comps)
#[print(df3[(df3['compound']==prob_co)]['isotopolog'].count()) for prob_co in prob_comps]

In [ ]:
df3[(df3['compound']==301)]['isotopolog'].count()/2

In [ ]:
df3[df3['compound']==351]['isotopolog'].count()

In [ ]:
#info about the data

def percent(part, whole):
    try:
        return 100 * float(part) / float(whole)
    except ZeroDivisionError:
        return 0
    
for prob_co in prob_comps:
    numm_scans=(df3['scan.no'].max())
    num_comp=df3[df3['compound']==prob_co]['isotopolog'].count()/2

    print('compound: '+str(prob_co))
    print('number of scans: ' +str(numm_scans))
    print('number of compounds:')
    print(num_comp)
    #calculate percent of missing data
    perc=100 - percent(num_comp,numm_scans)
    print(f'missing {perc:.2f} % of the data' )
    print('___________')

In [ ]:
df3['isotopolog']=='13C'

In [ ]:
#find scans with missing data (number of isotopologues M0 and 13C is not equal)
# for  each_scan in scans:
#     if count(isotopolog=='13C') != count(isotopolog=='M0')
#     remove line
for prob_co in prob_comps:
    print('________ Compound: '+str(prob_co))
    for each_scan in range(1, df3['scan.no'].max()):
        sum13c=(df3[(df3['scan.no']==each_scan) & (df3['compound']==prob_co) & (df3['isotopolog']=='13C')]['compound'].count())
        #print(sum13c)
        sumM0=(df3[(df3['scan.no']==each_scan) & (df3['compound']==prob_co) & (df3['isotopolog']=='M0')]['compound'].count())
        if sum13c!=sumM0:
            print('Number of scan: ' +str(each_scan)+ '; there is ' + str(sum13c) +' 13C and ' + str(sumM0) + ' M0') 
        #df3.drop(sum13c!=sumM0)



In [ ]:
df3[(df3['scan.no']==each_scan) & (df3['compound']==prob_co) & (df3['isotopolog']=='13C')]

In [52]:
# group by scan number 
pairs = df3.groupby(['scan.no','compound'])['isotopolog'].size().reset_index(name='counts')

pairs_to_keep = pairs[pairs['counts'] == 2].drop(columns='counts')
df3=df3.merge(pairs_to_keep[['scan.no', 'compound']], on=['scan.no', 'compound'], how='inner')

# create lists to delete  
# to_del_scanno = pairs[(pairs['counts'] != 2)]['scan.no'].values
# to_del_compound = pairs[(pairs['counts'] !=2 )]['compound'].values

# to_keep_scanno = pairs[(pairs['counts'] == 2)]['scan.no'].values
# to_keep_compound = pairs[(pairs['counts'] ==2 )]['compound'].values

# keep only nesessary data in dataframe
#df3_clean = df3[~(df3['scan.no'].isin(to_del_scanno) & df3['compound'].isin(to_del_compound))]
#df3_clean.info()
#df3_clean = df3[(df3['scan.no'].isin(to_keep_scanno) & df3['compound'].isin(to_keep_compound))]

In [53]:
pairs_to_keep = pairs[pairs['counts'] == 2].drop(columns='counts')

In [57]:
df3.merge(pairs_to_keep[['scan.no', 'compound']], on=['scan.no', 'compound'], how='inner')


,filename,scan.no,time.min,compound,isotopolog,ions.incremental,tic,it.ms,intensity,resolution,...,peakNoise,mzMeasured,basePeakIntensity,rawOvFtT,intensCompFactor,agc,agcTarget,microscans,numberLockmassesFound,analyzerTemperature
0,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,1,0.010,299,M0,36994.771415,2683131.8,504.158,124206.945312,35000,...,58.977,299.27286,196682.828125,1243409.6,1.0779,0,1000000,5,0,44.33
1,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,1,0.010,299,13C,9535.112631,2683131.8,504.158,32210.152344,35000,...,59.340,300.27655,196682.828125,1243409.6,1.0779,0,1000000,5,0,44.33
2,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,1,0.010,301,M0,22372.070625,2683131.8,504.158,76039.710938,35000,...,59.705,301.28867,196682.828125,1243409.6,1.0779,0,1000000,5,0,44.33
3,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,1,0.010,301,13C,5426.136742,2683131.8,504.158,18554.662109,35000,...,60.068,302.29196,196682.828125,1243409.6,1.0779,0,1000000,5,0,44.33
4,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,1,0.010,313,M0,53861.822996,2683131.8,504.158,196682.828125,35000,...,64.145,313.28851,196682.828125,1243409.6,1.0779,0,1000000,5,0,44.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54427,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,3163,89.904,349,13C,3564.774333,3131332.5,447.790,15843.203125,35000,...,78.071,350.25583,249413.593750,1291803.0,1.0934,0,1000000,5,0,44.49
54428,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,3163,89.904,351,M0,40356.718997,3131332.5,447.790,179156.000000,35000,...,77.982,351.26779,249413.593750,1291803.0,1.0934,0,1000000,5,0,44.49
54429,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,3163,89.904,351,13C,11125.329628,3131332.5,447.790,49332.894531,35000,...,77.894,352.27118,249413.593750,1291803.0,1.0934,0,1000000,5,0,44.49
54430,94_Cholesterol_Nice-SBL1-85_vs_Nice-Comm-84_12...,3163,89.904,353,M0,40317.457628,3131332.5,447.790,178574.843750,35000,...,77.805,353.28320,249413.593750,1291803.0,1.0934,0,1000000,5,0,44.49


In [54]:
pairs_to_keep

,scan.no,compound
0,1,299
1,1,301
2,1,313
3,1,325
4,1,327
...,...,...
27604,3163,325
27605,3163,327
27606,3163,349
27607,3163,351


In [ ]:
df3[(df3['scan.no']==1)]

In [ ]:
len(df3[(df3['scan.no'].isin(to_keep_scanno) & df3['compound'].isin(to_keep_compound))])

In [ ]:
len(to_keep_compound)

In [ ]:
len(to_keep_scanno)


In [ ]:
df3[(df3['scan.no'].isin(to_keep_scanno) & df3['compound'].isin(to_keep_compound))].info()

In [ ]:
df3[(df3['scan.no'].isin(to_del_scanno) & df3['compound'].isin(to_del_compound))].info()

In [ ]:
pairs

In [ ]:
df3[~(df3['scan.no'].isin(to_del_scanno) & df3['compound'].isin(to_del_compound))].info()

In [ ]:
df3.info()

In [ ]:
df3[(df3['scan.no'].isin(to_del_scanno) & df3['compound'].isin(to_del_compound))].info()

In [ ]:
len(to_del_scanno)

In [ ]:
set(to_del_compound)

In [ ]:
#finalising
def percent(part, whole):
    try:
        return 100 * float(part) / float(whole)
    except ZeroDivisionError:
        return 0

dfs_list=[df1,df2,df3,df4]
i=1
def find_prob_comp(dfs_list):
    for df in dfs_list:
        print('___Working on file #'+str(i) )
        i=i+1
        #find problematic compunds
        compounds=df['compound'].unique()
        #print(compounds)
        #print compound if its quantity does not match to number of scans
        prob_comps=[comp for comp in compounds if df[(df['compound']==comp)]['isotopolog'].count()/2 != df['scan.no'].max()]
        #print(prob_comps)
        
        # for each problmatic compound print number of scans and number of compounds   
        for prob_co in prob_comps:
            numm_scans=(df['scan.no'].max())
            num_comp=df[df['compound']==prob_co]['isotopolog'].count()/2
            #calculate percent of missing data
            perc = 100 - percent(num_comp,numm_scans)
            print('Problematic compound: '+str(prob_co) +'; No of scans/compounds: ' +str(numm_scans)+' / '  +str(num_comp) )
            print(f'   missing {perc:.2f} % of the data' )

In [ ]:
# new version via lists to keep
def delete_unpaired(dfs_list):
    dfs_list_new=[]
    for df in dfs_list:
        # group by scan number 
        pairs = df.groupby(['scan.no','compound'])['isotopolog'].size().reset_index(name='counts')

        # lists data to keep 
        to_keep_scanno = pairs[(pairs['counts'] == 2)]['scan.no'].values
        to_keep_compound = pairs[(pairs['counts'] ==2 )]['compound'].values

        # keep only nesessary data in dataframe
        df = df[(df['scan.no'].isin(to_keep_scanno) & df['compound'].isin(to_keep_compound))]
        dfs_list_new.append(df)l
        print('Unpaired lines are deleted')
    return dfs_list_new


In [ ]:
#old version via lists to delete
def delete_unpaired(dfs_list):
    #delete scans of compounds with unpaired isotopologs
    print('Starting to delete unpaired lines')
    dfs_list_new=[]
    for df in dfs_list:
        # group by scan number 
        pairs = df.groupby(['scan.no','compound'])['isotopolog'].size().reset_index(name='counts')

        # create lists to delete  
        to_del_scanno = pairs[(pairs['counts'] != 2)]['scan.no'].values
        to_del_compound = pairs[(pairs['counts'] !=2 )]['compound'].values

        # keep only nesessary data in dataframe
        df = df[~(df['scan.no'].isin(to_del_scanno) & df['compound'].isin(to_del_compound))]
        dfs_list_new.append(df)
        #yield df
        print('Unpaired lines are deleted')
    return dfs_list_new

In [ ]:
pairs[(pairs['counts'] !=2 )]['compound'].values

In [ ]:
df3.info()

In [ ]:
df3_clean

In [ ]:
odd_df['compound']!=odd_df['compound_s']

In [ ]:
list([odd_df['compound']!=odd_df['compound_s'].index])

In [ ]:
(df3['compound'] == df3['compound_s']) 